In [1]:
if 'first_run' not in globals():
    !pip install torch torchvision torchaudio
    !pip install torch torch-geometric
    first_run = False
else:
    pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
import hashlib
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.transforms import BaseTransform
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.data import Data
from torch_geometric.io import fs, read_txt_array
import torch_geometric.transforms as T
# from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.data import DataLoader
import math
from typing import Callable, List, Optional, Tuple, Dict
from torch import Tensor
import os
import requests
from torch_geometric.utils import coalesce, cumsum, one_hot, remove_self_loops
from sklearn.metrics import confusion_matrix
from torch.cuda.amp import GradScaler, autocast
import os.path as osp

import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import degree
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
from torch_geometric.nn import GINConv, global_mean_pool, JumpingKnowledge
from torch_geometric.nn import MessagePassing
from torch_geometric.nn.inits import reset
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dd-dga/__results__.html
/kaggle/input/dd-dga/DD.pt
/kaggle/input/dd-dga/DD_train_indices_dict.pkl
/kaggle/input/dd-dga/__notebook__.ipynb
/kaggle/input/dd-dga/__output__.json
/kaggle/input/dd-dga/custom.css
/kaggle/input/enzymes-dga/__results__.html
/kaggle/input/enzymes-dga/ENZYMES.pt
/kaggle/input/enzymes-dga/__notebook__.ipynb
/kaggle/input/enzymes-dga/__output__.json
/kaggle/input/enzymes-dga/ENZYMES_train_indices_dict.pkl
/kaggle/input/enzymes-dga/custom.css
/kaggle/input/mutag-dga/MUTAG.pt
/kaggle/input/mutag-dga/MUTAG_train_indices_dict.pkl
/kaggle/input/mutag-dga/__results__.html
/kaggle/input/mutag-dga/__notebook__.ipynb
/kaggle/input/mutag-dga/__output__.json
/kaggle/input/mutag-dga/custom.css
/kaggle/input/nci1-dga/NCI1.pt
/kaggle/input/nci1-dga/__results__.html
/kaggle/input/nci1-dga/NCI1_train_indices_dict.pkl
/kaggle/input/nci1-dga/__notebook__.ipynb
/kaggle/input/nci1-dga/__output__.json
/kaggle/input/nci1-dga/custom.css
/kaggle/input/nci109-dga/__results__

In [3]:
class IDAddingDataset:
    def __init__(self, dataset, attr_func):
        self.dataset = dataset
        self.attr_func = attr_func
        self.indices = np.arange(0, len(dataset.indices()))
    
    def __getitem__(self, idx):
        data = self.dataset[idx].clone()  # Clone the data to avoid modifying the original dataset
        data.id = self.indices[idx]
        data.num_edges = self.dataset[idx].edge_index.size(1)
        data.adj_mat = to_scipy_sparse_matrix(self.dataset[idx].edge_index, num_nodes=self.dataset[idx].num_nodes).toarray().astype(int)
        return data
    
    def __len__(self):
        return len(self.dataset)
class EmbeddingAddingDataset:
    def __init__(self, dataset, embedding_dict):
        self.dataset = dataset
        self.embedding_dict = embedding_dict
    
    def __getitem__(self, idx):
        data = self.dataset[idx].clone()  # Clone the data to avoid modifying the original dataset
        data.embedding = self.embedding_dict[self.dataset[idx].id]
        return data
    
    def __len__(self):
        return len(self.dataset)
class FinalAddingDataset:
    def __init__(self, Embedding_dataset, cos_sims, all_z_scores):
        self.dataset = Embedding_dataset
        self.cos_sims = cos_sims
        self.all_z_scores = all_z_scores
        
    
    def __getitem__(self, idx):
        data = self.dataset[idx].clone()  # Clone the data to avoid modifying the original dataset
        data.cos_sim = self.cos_sims[self.dataset[idx].id]
        data.z_score = self.all_z_scores[self.dataset[idx].id]
        return data
    
    def __len__(self):
        return len(self.dataset)

In [4]:
class TrainDatasetWrapper:
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __getitem__(self, idx):
        data = self.dataset[idx].clone()  # Clone to avoid modifying the original dataset
        return Data(edge_index=data.edge_index, x=data.x, y=data.y.item())  # Return only necessary attributes
    
    def __len__(self):
        return len(self.dataset)

In [5]:
class SubsetDataset:
    def __init__(self, full_dataset, indices):
        self.dataset = full_dataset
        self.indices = indices

    def __getitem__(self, idx):
        actual_idx = self.indices[idx]
        return self.dataset[actual_idx]

    def __len__(self):
        return len(self.indices)


In [6]:
class NormalizedDegree(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        deg = degree(data.edge_index[0], dtype=torch.float)
        deg = (deg - self.mean) / self.std
        data.x = deg.view(-1, 1)
        return data

In [7]:
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.bn1 = BatchNorm(16)
        self.conv2 = GCNConv(16, 32)
        self.bn2 = BatchNorm(32)
        self.conv3 = GCNConv(32, 64)
        self.bn3 = BatchNorm(64)
        self.conv4 = GCNConv(64, 128)
        self.bn4 = BatchNorm(128)
        self.fc1 = torch.nn.Linear(128, 64)
        self.fc2 = torch.nn.Linear(64, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply the first convolution and batch normalization
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = F.dropout(x, training=self.training)

        # Apply the second convolution and batch normalization
        x = F.relu(self.bn2(self.conv2(x, edge_index)))

        # Apply the third convolution and batch normalization
        x = F.relu(self.bn3(self.conv3(x, edge_index)))

        # Apply the fourth convolution and batch normalization
        x = F.relu(self.bn4(self.conv4(x, edge_index)))

        # Global mean pooling
        x = global_mean_pool(x, data.batch)

        # MLP for the final output
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x  # Return logits for use with nn.CrossEntropyLoss

In [8]:
# Taken from https://github.com/rusty1s/pytorch_geometric/blob/master/benchmark/kernel/gin.py.
class GIN(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, num_classes):
        super(GIN, self).__init__()
        self.conv1 = GINConv(Sequential(
            Linear(dataset[0].num_features, hidden),
            ReLU(),
            Linear(hidden, hidden),
            ReLU(),
            BN(hidden),
        ),
            train_eps=True)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(
                GINConv(Sequential(
                    Linear(hidden, hidden),
                    ReLU(),
                    Linear(hidden, hidden),
                    ReLU(),
                    BN(hidden),
                ),
                    train_eps=True))
        self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        for conv in self.convs:
            x = conv(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__

In [9]:
# Taken from https://github.com/rusty1s/pytorch_geometric/blob/master/benchmark/kernel/gin.py.
class GINWithJK(torch.nn.Module):
    def __init__(self, dataset, num_layers, hidden, mode='cat'):
        super(GINWithJK, self).__init__()
        self.conv1 = GINConv(Sequential(
            Linear(dataset[0].num_features, hidden),
            ReLU(),
            Linear(hidden, hidden),
            ReLU(),
            BN(hidden),
        ),
            train_eps=True)
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(
                GINConv(Sequential(
                    Linear(hidden, hidden),
                    ReLU(),
                    Linear(hidden, hidden),
                    ReLU(),
                    BN(hidden),
                ),
                    train_eps=True))
        self.jump = JumpingKnowledge(mode)
        if mode == 'cat':
            self.lin1 = Linear(num_layers * hidden, hidden)
        else:
            self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden, num_classes)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        for conv in self.convs:
            conv.reset_parameters()
        self.jump.reset_parameters()
        self.lin1.reset_parameters()
        self.lin2.reset_parameters()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        xs = [x]
        for conv in self.convs:
            x = conv(x, edge_index)
            xs += [x]
        x = self.jump(xs)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)

    def __repr__(self):
        return self.__class__.__name__


In [10]:
def split_data(name, data, train_indices_dict, all_or_class = 'class', characterization_method = 'random', percentile = 90):
    # Assuming 'data.__len__()' returns the total number of instances in the dataset
    all_indices = set(range(data.__len__()))

    # Assuming 'train_val' contains the combined set of training and validation indices
    train_val = set(train_indices_dict[name][all_or_class][characterization_method][percentile])
    # Convert train_val to a numpy array if not already
    train_val_indices = np.array(list(train_val))
    test_set = all_indices - train_val
    test_indices = np.array(list(test_set))
    # Randomly select 80% of the indices for training
    train_indices = np.random.choice(train_val_indices, size=int(0.8 * len(train_val_indices)), replace=False)

    # Convert train_indices back to a set to perform set subtraction
    train_set = set(train_indices)

    # The remaining 20% will be for validation
    val_indices = np.array(list(set(train_val) - train_set))
    indices = {
    'train': train_indices,  # fill with your training indices
    'val': val_indices,    # fill with your validation indices
    'test': test_indices    # fill with your testing indices
    }
    train_dataset = SubsetDataset(data, indices['train'])
    val_dataset = SubsetDataset(data, indices['val'])
    test_dataset = SubsetDataset(data, indices['test'])
    
    return train_dataset, test_dataset, val_dataset

In [11]:
def calculate_global_metrics(all_labels, all_preds):
    # Determine the number of unique classes
    unique_classes = np.unique(all_labels)
    cm = confusion_matrix(all_labels, all_preds)
    tp = np.diag(cm).sum()
    pop = len(all_preds)  # Total number of predictions
    
    if len(unique_classes) == 2:
        # Binary classification
        tn, fp, fn, tp = cm.ravel()
    else:
        # Multi-class classification
        fp = cm.sum(axis=0) - np.diag(cm)  # False positives are summed over columns, exclude true positives
        fn = cm.sum(axis=1) - np.diag(cm)  # False negatives are summed over rows, exclude true positives
        tn = None  # True negatives are not defined the same way in multi-class scenarios

        # Sum FP and FN for overall metrics
        fp = fp.sum()
        fn = fn.sum()

    # Calculate accuracy, handling cases where TN is not used
    acc = tp / pop

    return acc, tn, fp, fn, tp

In [12]:
def gnn_train(train_loader, model, optimizer, device):
    model.train()

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        optimizer.step()
        return loss


# Get acc. of GNN model.
def gnn_test(loader, model, device):
    model.eval()
    all_preds = []
    all_labels = []
    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        all_preds.extend(pred.cpu().numpy())  # Store predictions
        all_labels.extend(data.y.cpu().numpy())
    return calculate_global_metrics(all_labels, all_preds)
def gnn_evaluation(
    gnn, 
    ds_name, 
    dataset, 
    train_indices_dict, 
    _all_or_class, 
    method, 
    layers, 
    hidden, 
    num_classes,
    max_num_epochs=200, 
    batch_size=128, 
    start_lr=0.01, 
    min_lr = 0.000001, 
    factor=0.5, 
    patience=5,
    num_repetitions=10, 
    all_std=True
):
    # One-hot degree if node labels are not available.
    # The following if clause is taken from  https://github.com/rusty1s/pytorch_geometric/blob/master/benchmark/kernel/datasets.py.
#     if dataset.data.x is None:
#         max_degree = 0
#         degs = []
#         for data in dataset:
#             degs += [degree(data.edge_index[0], dtype=torch.long)]
#             max_degree = max(max_degree, degs[-1].max().item())

#         if max_degree < 1000:
#             dataset.transform = T.OneHotDegree(max_degree)
#         else:
#             deg = torch.cat(degs, dim=0).to(torch.float)
#             mean, std = deg.mean().item(), deg.std().item()
#             dataset.transform = NormalizedDegree(mean, std)

#     # Set device.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    test_accuracies_all = []
    test_accuracies_complete = []

    # Test acc. over all folds.
    test_accuracies = []


    best_val_acc = 0.0
    best_test = 0.0

    # Split data.
    train_dataset, test_dataset, val_dataset = split_data(
        ds_name,
        dataset, 
        train_indices_dict, 
        all_or_class = _all_or_class, 
        characterization_method = method, 
        percentile = pctl
    )
    
    # Prepare batching.
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    if gnn == 'GIN':
        model = GIN(dataset, layers, hidden, num_classes).to(device)
        model.reset_parameters()
    else:
        model = GINWithJK(dataset, layers, hidden, num_classes).to(device)
        model.reset_parameters()
        
    optimizer = torch.optim.Adam(model.parameters(), lr=start_lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min',
        factor=factor, 
        patience=patience,
        min_lr=0.0000001
    )
    results_dict = {'train': {'train_loss': [], 'val_acc': []}, 'test': {'acc': -1.0, 'confusion': {}}}
    print('Epoch/val accuracy/lr', end = ': ')
    for epoch in range(1, max_num_epochs + 1):
        lr = scheduler.optimizer.param_groups[0]['lr']
        train_loss = gnn_train(train_loader, model, optimizer, device)
        val_acc = gnn_test(val_loader, model, device)[0]
        results_dict['train']['train_loss'].append(train_loss)
        results_dict['train']['val_acc'].append(val_acc)
        scheduler.step(val_acc)
        print(f'{epoch}/{val_acc:.4f}/{lr}', end = ' | ')
        if lr < min_lr:
            break

    acc, tn, fp, fn, tp = gnn_test(test_loader, model, device)

    results_dict['test']['acc'] = acc
    results_dict['test']['confusion'] = {'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn}
    print(f'Accuracy: {acc:.4f}')
    return results_dict
        # Break if learning rate is smaller 10**-6.
        
    


In [13]:
# def my_gcn_train(name, data, train_indices_dict, method, _all_or_class, pctl, num_epochs):
#     num_features = data[0].num_features
#     results_dict = {}
#     train_dataset, test_dataset, val_dataset = split_data(
#         name,
#         data, 
#         train_indices_dict, 
#         all_or_class = _all_or_class, 
#         characterization_method = method, 
#         percentile = pctl
#     )
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model = GCN(num_features=num_features, num_classes=6).to(device)
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
#     train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers = 4, pin_memory = True)
#     val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers = 4, pin_memory = True)
#     test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

#     # Initialize the loss function
#     criterion = nn.CrossEntropyLoss()
#     # Training loop
#     scaler = GradScaler()

#     def train():
#         model.train()
#         total_loss = 0
#         for data in train_loader:
#             data = data.to(device)
#             optimizer.zero_grad()

#             with autocast():
#                 out = model(data)  # outputs logits
#                 loss = criterion(out, data.y)

#             scaler.scale(loss).backward()
#             scaler.step(optimizer)
#             scaler.update()

#             total_loss += loss.item()

#         return total_loss / len(train_loader)

#     def evaluate(loader):
#         model.eval()
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for data in loader:
#                 data = data.to(device)
#                 out = model(data)
#                 pred = out.argmax(dim=1)
#                 correct += int((pred == data.y).sum())
#                 total += data.y.size(0)
#         return correct / total
#     # Assuming you have a DataLoader for your test data similar to train_loader

#     def test():
#         model.eval()  # Set the model to evaluation mode
#         all_preds = []
#         all_labels = []

#         with torch.no_grad():  # Inference without gradient calculation
#             for data in test_loader:
#                 data = data.to(device)
#                 out = model(data)  # Forward pass
#                 pred = out.argmax(dim=1)  # Get the index of the max log-probability
#                 all_preds.extend(pred.cpu().numpy())  # Store predictions
#                 all_labels.extend(data.y.cpu().numpy())  # Store actual labels

#         # Calculate confusion matrix
#         acc, tn, fp, fn, tp = calculate_global_metrics(all_labels, all_preds)
#         return acc, tn, fp, fn, tp
    
#     results_dict = {'train': {'train_loss': [], 'val_acc': []}, 'test': {'acc': -1.0, 'confusion': {}}}
#     print(f'Training GCN on {name} for {num_epochs} epochs:')
#     print('Epoch/train loss/val accuracy', end = ': ')
#     for epoch in range(num_epochs):
#         train_loss = train()
#         val_accuracy = evaluate(val_loader)
#         results_dict['train']['train_loss'].append(train_loss)
#         results_dict['train']['val_acc'].append(val_accuracy)
#         print(f'{epoch}/{train_loss:.4f}/{val_accuracy:.4f}', end = ' | ')
#         scheduler.step()
#     acc, tn, fp, fn, tp = test()
#     results_dict['test']['acc'] = acc
#     results_dict['test']['confusion'] = {'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn}
#     print(f'Accuracy: {acc:.4f}')
#     return results_dict

In [14]:
dataset_list = ['enzymes', 'dd', 'nci1', 'nci109', 'proteins', 'mutag']
char_methods = ['random', 'graph_order', 'cos_sim']
percentiles = [x for x in range(40, 91, 10)]

In [15]:
if 'dataset_dict' not in globals():
    dataset_dict = {}
    train_indices_dict = {}
    for dataset in dataset_list:
        with open(f'/kaggle/input/{dataset}-dga/{dataset.upper()}.pt', 'rb') as file:
            dataset_dict[dataset] = TrainDatasetWrapper(torch.load(file))
        with open(f'/kaggle/input/{dataset}-dga/{dataset.upper()}_train_indices_dict.pkl', 'rb') as file:
            train_indices_dict[dataset] = pickle.load(file)
    num_classes_dict = {dataset: 2 if dataset != 'enzymes' else 6 for dataset in dataset_list}

In [16]:
results_dict = {}
for name, dataset in dataset_dict.items():
    results_dict[name] = {}
    for char_method in char_methods:
        results_dict[name][char_method] = {}
        for pctl in percentiles:
            results_dict[name][char_method][pctl] = {}
            for pop_method in ['class']:
                print(f'Evaluating {name} dataset using {char_method} method with a percentile of {pctl} against {pop_method} members')
                results_dict[name][char_method][pctl][pop_method] = gnn_evaluation(
                    "GIN", 
                    ds_name = name, 
                    dataset = dataset, 
                    train_indices_dict = train_indices_dict, 
                    _all_or_class = pop_method, 
                    method = char_method, 
                    layers = 4, 
                    hidden = 64, 
                    num_classes = num_classes_dict[name]
                )


Evaluating enzymes dataset using random method with a percentile of 40 against class members
Epoch/val accuracy/lr: 

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


1/0.0833/0.01 | 2/0.1875/0.01 | 3/0.1667/0.01 | 4/0.1458/0.01 | 5/0.1458/0.01 | 6/0.1875/0.01 | 7/0.2083/0.01 | 8/0.1875/0.005 | 9/0.2500/0.005 | 10/0.2500/0.005 | 11/0.2500/0.005 | 12/0.2708/0.005 | 13/0.2292/0.005 | 14/0.2083/0.0025 | 15/0.2292/0.0025 | 16/0.2083/0.0025 | 17/0.2083/0.0025 | 18/0.2292/0.0025 | 19/0.2292/0.0025 | 20/0.2292/0.00125 | 21/0.2292/0.00125 | 22/0.2500/0.00125 | 23/0.2500/0.00125 | 24/0.2500/0.00125 | 25/0.1875/0.00125 | 26/0.1875/0.000625 | 27/0.1667/0.000625 | 28/0.1458/0.000625 | 29/0.1667/0.000625 | 30/0.1667/0.000625 | 31/0.2292/0.000625 | 32/0.2083/0.0003125 | 33/0.2292/0.0003125 | 34/0.2292/0.0003125 | 35/0.2292/0.0003125 | 36/0.2083/0.0003125 | 37/0.2083/0.0003125 | 38/0.2083/0.00015625 | 39/0.1875/0.00015625 | 40/0.1875/0.00015625 | 41/0.1875/0.00015625 | 42/0.1875/0.00015625 | 43/0.1875/0.00015625 | 44/0.1875/7.8125e-05 | 45/0.1875/7.8125e-05 | 46/0.1875/7.8125e-05 | 47/0.1875/7.8125e-05 | 48/0.1667/7.8125e-05 | 49/0.1875/7.8125e-05 | 50/0.1667/3.90

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


1/0.1167/0.01 | 2/0.1500/0.01 | 3/0.1500/0.01 | 4/0.1500/0.01 | 5/0.1500/0.01 | 6/0.1500/0.01 | 7/0.1500/0.01 | 8/0.1500/0.005 | 9/0.1667/0.005 | 10/0.1500/0.005 | 11/0.1167/0.005 | 12/0.1333/0.005 | 13/0.1500/0.005 | 14/0.1667/0.0025 | 15/0.1333/0.0025 | 16/0.2000/0.0025 | 17/0.2333/0.0025 | 18/0.2000/0.0025 | 19/0.2000/0.0025 | 20/0.1667/0.00125 | 21/0.1333/0.00125 | 22/0.1333/0.00125 | 23/0.1500/0.00125 | 24/0.1667/0.00125 | 25/0.1667/0.00125 | 26/0.1333/0.000625 | 27/0.1167/0.000625 | 28/0.1167/0.000625 | 29/0.1167/0.000625 | 30/0.1167/0.000625 | 31/0.1167/0.000625 | 32/0.1167/0.0003125 | 33/0.0833/0.0003125 | 34/0.0833/0.0003125 | 35/0.0833/0.0003125 | 36/0.1000/0.0003125 | 37/0.0833/0.0003125 | 38/0.0667/0.0003125 | 39/0.0667/0.0003125 | 40/0.0667/0.0003125 | 41/0.0667/0.0003125 | 42/0.1000/0.0003125 | 43/0.1167/0.0003125 | 44/0.1167/0.0003125 | 45/0.1167/0.00015625 | 46/0.1333/0.00015625 | 47/0.1500/0.00015625 | 48/0.1500/0.00015625 | 49/0.1500/0.00015625 | 50/0.1500/0.00015625 

...

1/0.0000/0.01 | 2/0.7407/0.01 | 3/0.7407/0.01 | 4/0.7407/0.01 | 5/0.7407/0.01 | 6/0.7407/0.01 | 7/0.7407/0.01 | 8/0.7407/0.005 | 9/0.7407/0.005 | 10/0.7407/0.005 | 11/0.7407/0.005 | 12/0.7407/0.005 | 13/0.7407/0.005 | 14/0.7407/0.0025 | 15/0.7407/0.0025 | 16/0.7407/0.0025 | 17/0.7407/0.0025 | 18/0.7407/0.0025 | 19/0.7407/0.0025 | 20/0.7407/0.00125 | 21/0.7407/0.00125 | 22/0.7407/0.00125 | 23/0.7407/0.00125 | 24/0.7407/0.00125 | 25/0.7407/0.00125 | 26/0.7407/0.000625 | 27/0.7407/0.000625 | 28/0.7407/0.000625 | 29/0.7407/0.000625 | 30/0.7407/0.000625 | 31/0.7407/0.000625 | 32/0.7407/0.0003125 | 33/0.7407/0.0003125 | 34/0.7407/0.0003125 | 35/0.7407/0.0003125 | 36/0.7407/0.0003125 | 37/0.7037/0.0003125 | 38/0.7037/0.00015625 | 39/0.7037/0.00015625 | 40/0.7037/0.00015625 | 41/0.6667/0.00015625 | 42/0.6667/0.00015625 | 43/0.6667/0.00015625 | 44/0.6296/7.8125e-05 | 45/0.6296/7.8125e-05 | 46/0.6296/7.8125e-05 | 47/0.6296/7.8125e-05 | 48/0.6296/7.8125e-05 | 49/0.5926/7.8125e-05 | 50/0.5926/3.90

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


1/0.7097/0.01 | 2/0.7097/0.01 | 3/0.7097/0.01 | 4/0.7097/0.01 | 5/0.7097/0.01 | 6/0.7097/0.01 | 7/0.7097/0.01 | 8/0.7097/0.005 | 9/0.7097/0.005 | 10/0.7097/0.005 | 11/0.7097/0.005 | 12/0.7097/0.005 | 13/0.7097/0.005 | 14/0.7097/0.0025 | 15/0.7097/0.0025 | 16/0.6452/0.0025 | 17/0.6452/0.0025 | 18/0.6452/0.0025 | 19/0.6452/0.0025 | 20/0.7097/0.0025 | 21/0.7097/0.0025 | 22/0.7097/0.0025 | 23/0.7097/0.00125 | 24/0.7097/0.00125 | 25/0.7097/0.00125 | 26/0.7097/0.00125 | 27/0.7097/0.00125 | 28/0.7097/0.00125 | 29/0.7097/0.000625 | 30/0.7097/0.000625 | 31/0.6774/0.000625 | 32/0.7097/0.000625 | 33/0.7097/0.000625 | 34/0.7097/0.000625 | 35/0.7097/0.0003125 | 36/0.7097/0.0003125 | 37/0.7097/0.0003125 | 38/0.7097/0.0003125 | 39/0.7097/0.0003125 | 40/0.6774/0.0003125 | 41/0.6774/0.00015625 | 42/0.7097/0.00015625 | 43/0.7097/0.00015625 | 44/0.7097/0.00015625 | 45/0.7097/0.00015625 | 46/0.7097/0.00015625 | 47/0.7097/7.8125e-05 | 48/0.7097/7.8125e-05 | 49/0.7097/7.8125e-05 | 50/0.7097/7.8125e-05 | 51/

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


1/0.6765/0.01 | 2/0.6765/0.01 | 3/0.6765/0.01 | 4/0.6765/0.01 | 5/0.6765/0.01 | 6/0.6765/0.01 | 7/0.6765/0.01 | 8/0.6765/0.005 | 9/0.6765/0.005 | 10/0.6765/0.005 | 11/0.6765/0.005 | 12/0.6765/0.005 | 13/0.6765/0.005 | 14/0.6765/0.0025 | 15/0.6765/0.0025 | 16/0.6765/0.0025 | 17/0.5588/0.0025 | 18/0.5000/0.0025 | 19/0.4706/0.0025 | 20/0.4412/0.0025 | 21/0.4412/0.0025 | 22/0.4412/0.0025 | 23/0.4412/0.0025 | 24/0.4412/0.0025 | 25/0.4118/0.0025 | 26/0.4118/0.0025 | 27/0.4118/0.0025 | 28/0.3824/0.0025 | 29/0.3824/0.0025 | 30/0.3824/0.0025 | 31/0.3824/0.0025 | 32/0.3824/0.0025 | 33/0.3824/0.0025 | 34/0.3824/0.0025 | 35/0.3824/0.00125 | 36/0.3824/0.00125 | 37/0.3824/0.00125 | 38/0.3824/0.00125 | 39/0.3824/0.00125 | 40/0.3824/0.00125 | 41/0.3824/0.000625 | 42/0.3235/0.000625 | 43/0.3824/0.000625 | 44/0.3824/0.000625 | 45/0.3824/0.000625 | 46/0.3824/0.000625 | 47/0.3824/0.000625 | 48/0.3824/0.000625 | 49/0.4412/0.0003125 | 50/0.4412/0.0003125 | 51/0.4412/0.0003125 | 52/0.4412/0.0003125 | 53/0.44

In [17]:
import pickle
with open('/kaggle/working/results_dict.pkl', 'wb') as f:
    pickle.dump(results_dict, f)




In [18]:
print(results_dict['enzymes']['random'])

{40: {'class': {'train': {'train_loss': [tensor(1.8107, grad_fn=<NllLossBackward0>), tensor(1.7770, grad_fn=<NllLossBackward0>), tensor(1.8008, grad_fn=<NllLossBackward0>), tensor(1.6791, grad_fn=<NllLossBackward0>), tensor(1.6892, grad_fn=<NllLossBackward0>), tensor(1.6432, grad_fn=<NllLossBackward0>), tensor(1.6070, grad_fn=<NllLossBackward0>), tensor(1.6401, grad_fn=<NllLossBackward0>), tensor(1.5047, grad_fn=<NllLossBackward0>), tensor(1.5453, grad_fn=<NllLossBackward0>), tensor(1.5768, grad_fn=<NllLossBackward0>), tensor(1.5200, grad_fn=<NllLossBackward0>), tensor(1.5058, grad_fn=<NllLossBackward0>), tensor(1.5276, grad_fn=<NllLossBackward0>), tensor(1.5500, grad_fn=<NllLossBackward0>), tensor(1.5535, grad_fn=<NllLossBackward0>), tensor(1.4741, grad_fn=<NllLossBackward0>), tensor(1.5361, grad_fn=<NllLossBackward0>), tensor(1.4973, grad_fn=<NllLossBackward0>), tensor(1.4607, grad_fn=<NllLossBackward0>), tensor(1.4425, grad_fn=<NllLossBackward0>), tensor(1.4440, grad_fn=<NllLossBack